In [1]:
:opt no-lint

In [2]:
-- Prolog term
data Term = V String        -- logic variable.       String 대문자로 시작
          | T String [Term] -- atom when empty list. String 소문자로 시작
      deriving (Eq,Ord,Show)

In [3]:
import Data.List
-- Prolog term처럼 보이게 좀더 예쁘게 출력할 수 있는 문자열로
pp (V x) = x
pp (T a []) = a
pp (T f ts) = f ++ (paren . concat . intersperse "," $ map pp ts )

paren s = "("++s++")"

In [4]:
-- 변수 X
V "X" 
pp(V "X")

V "X"

"X"

In [5]:
-- 아톰 int 
T "int" []
pp(T "int" [])

T "int" []

"int"

In [6]:
-- 복합항 f(int,int)
T "f" [T "int" [], T "int" []]
pp(T "f" [T "int" [], T "int" []])

T "f" [T "int" [],T "int" []]

"f(int,int)"

일치화 규칙

$\displaystyle
[a=a,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[f(t_1,\ldots,t_n)=f(t_1',\ldots,t_n'),~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[t_1=t_1',\cdots,t_n=t_n',u_2=u_2',~\cdots~],\sigma \\
[X=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[X=t,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
\{X\mapsto t\}\,[u_2=u_2',~\cdots~],\{X\mapsto t\}\uplus\sigma \qquad (X~\text{does not occur in}~ t) \\
[t=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[X=t,u_2=u_2',~\cdots~],\sigma
$

$\{X\mapsto t\}\,[u_1=s_1,\cdots,u_n=s_n] = \big[\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1,~\cdots,~\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1\big]$

$\{X\mapsto t\}\uplus\{X_1\mapsto t_1,~\cdots~,X_n\mapsto t_n \} = \big\{X\mapsto t, x_1\mapsto \{X\mapsto t\}t_1,~\cdots~,X_n\mapsto \{X\mapsto t\}t_n \big\}
$

In [7]:
occurs :: String -> Term -> Bool
occurs x (V x') = x==x'
occurs x (T _ ts) = any (occurs x) ts
-- occurs x (T _ ts) = or [occurs x t | t<-ts]

In [8]:
occurs "X" (T "f" [T "int" [], V "Y"])
occurs "X" (T "f" [T "int" [], V "X"])

False

True

In [9]:
subst :: (String,Term) -> Term -> Term
subst (x,t) u@(V y)  | x == y    = t
                     | otherwise = u
subst (x,t) (T f us) = T f [subst (x,t) u | u<-us]

In [10]:
-- {X |-> f(int,int)} f(X,X) ~~> f(f(int,int),f(int,int))
int = T "int" []

tm1 = T"f"[V"X",V"X"]
tm2 = subst ("X",T"f"[int,int]) (T"f"[V"X",V"X"])

pp tm1
pp tm2

"f(X,X)"

"f(f(int,int),f(int,int))"

일치화 규칙

$\displaystyle
[a=a,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[f(t_1,\ldots,t_n)=f(t_1',\ldots,t_n'),~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[t_1=t_1',\cdots,t_n=t_n',u_2=u_2',~\cdots~],\sigma \\
[X=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[X=t,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
\{X\mapsto t\}\,[u_2=u_2',~\cdots~],\{X\mapsto t\}\uplus\sigma \qquad (X~\text{does not occur in}~ t) \\
[t=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[X=t,u_2=u_2',~\cdots~],\sigma
$

$\{X\mapsto t\}\,[u_1=s_1,\cdots,u_n=s_n] = \big[\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1,~\cdots,~\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1\big]$

$\{X\mapsto t\}\uplus\{X_1\mapsto t_1,~\cdots~,X_n\mapsto t_n \} = \big\{X\mapsto t, x_1\mapsto \{X\mapsto t\}t_1,~\cdots~,X_n\mapsto \{X\mapsto t\}t_n \big\}
$

In [11]:
zip [1,2,3,4] [11,12,13,14]

[(1,11),(2,12),(3,13),(4,14)]

In [12]:
type Eqn = (Term,Term)       -- t1 = t2
type Subst = [(String,Term)] -- { X1 |-> t1, ..., Xn |-> tn }

-- 일치화 알고리듬을 위 규칙에 따라 작성
unif :: ([Eqn],Subst) -> ([Eqn],Subst)
unif ((T f ts, T g us): es, s)
       | f==g && length ts==length us  = unif (zip ts us ++ es, s)
unif ((V x   , V x'  ): es, s) | x==x' = unif (es, s)
unif ((V x   , t     ): es, s) | not(x `occurs` t) = unif (es', (x,t):s')
  where es' = [(subst (x,t) t1, subst (x,t) t2) | (t1,t2)<-es]
        s' = [(x',subst (x,t) t') | (x',t')<-s]
unif ((t     , V x   ): es, s) = unif ((V x,t):es, s)
unif (es, s) = (es, s)

In [13]:
t1 = T"c"[ V"X", T"c"[T"3"[],V"X"] ] -- c(X,c(3,X))
t2 = T"c"[ T"2"[], V"Y" ]            -- c(2,Y)

unif ([(t1,t2)], [])

([],[("Y",T "c" [T "3" [],T "2" []]),("X",T "2" [])])

In [14]:
t1 = T"c"[ V"X", T"c"[T"3"[],V"X"] ] -- c(X,c(3,X))
t2 = T"c"[ T"2"[], T"c"[V"X",V"Y"] ] -- c(2,c(X,X))

unif ([(t1,t2)], [])

([(T "3" [],T "2" []),(T "2" [],V "Y")],[("X",T "2" [])])

## 규칙 실행
규칙 실행은 여러 세트의 연립방정식을 만들어냄

규칙 실행의 결과 타입은 `[ [Eqn] ]`

In [15]:
-- Prolog에서 =
(.=) :: Term -> Term -> [ [Eqn] ]
t1 .= t2  =  [ [(t1,t2)] ]

-- Prolog에서 ,
(.&) :: [ [Eqn] ] -> [ [Eqn] ] -> [ [Eqn] ]
ess1 .& ess2  = [es1++es2  | es1<-ess1, es2<-ess2]

-- Prolog에서 ;
(.|) :: [ [Eqn] ] -> [ [Eqn] ] -> [ [Eqn] ]
ess1 .| ess2  = ess1 ++ ess2

### 필요에 따라서 새로운 논리변수를 만들어낼 수 있어야

Prolog 규칙을 하스켈로 옮겨 실행하기 위해 지난 번 UnifPrologTerm 노트북에서 작성한 함수의 타입은 다음과 같았다.
```haskell
likes  :: (Term, Term) -> [[Eqn]]
couple :: (Term, Term) -> [[Eqn]]
```

새로운 논리변수를 만드는 한가지 방법은 숫자를 매겨 "_0", "_1", "_2" 이런 식으로 겹치지 않게 변수를 나가는 것이다.

새로운 변수를 공급해줄 수 있는 상자에 해당하는 타입생성자를 `m`이라고 하면 대략 다음과 같은 타입이 되리라 예상해 볼 수 있다.

```haskell
father  :: Monad m => (Term, Term) -> m [ [Eqn] ]
grandpa :: Monad m => (Term, Term) -> m [ [Eqn] ]
```

In [16]:
t1 .=. t2 = return $ t1.=t2
m1 .&. m2 = (.&) <$> m1 <*> m2
m1 .|. m2 = (.|) <$> m1 <*> m2

In [17]:
:type (.=)
:type (.&)
:type (.|)

(.=) :: Term -> Term -> [[Eqn]]

(.&) :: [[Eqn]] -> [[Eqn]] -> [[Eqn]]

(.|) :: [[Eqn]] -> [[Eqn]] -> [[Eqn]]

In [18]:
:type (.=.)
:type (.&.)
:type (.|.)

(.=.) :: forall (m :: * -> *). Monad m => Term -> Term -> m [[Eqn]]

(.&.) :: forall (f :: * -> *). Applicative f => f [[Eqn]] -> f [[Eqn]] -> f [[Eqn]]

(.|.) :: forall (f :: * -> *). Applicative f => f [[Eqn]] -> f [[Eqn]] -> f [[Eqn]]

In [19]:
tim = T "tim" []
bob = T "bob" []
jane = T "jane" []
mary = T "mary" []
john = T "john" []
luke = T "luke" []
kate = T "kate" []

male(x) = (x.=.tim) .|.
          (x.=.bob) .|.
          (x.=.john)
          
female(x) = (x.=.jane) .|.
            (x.=.mary)

parent(p,c) = ( (p.=.tim) .&.(c.=.bob)  ) .|.
              ( (p.=.jane).&.(c.=.bob)  ) .|.
              ( (p.=.john).&.(c.=.tim)  ) .|.
              ( (p.=.mary).&.(c.=.tim)  ) .|.
              ( (p.=.luke).&.(c.=.john) ) .|.
              ( (p.=.kate).&.(c.=.john) )

father(f,c) = parent(f,c) .&. male(f)

In [20]:
solve ess = [unif (es,[]) | es<-ess]

In [21]:
:type male
:type female
:type parent
:type father

male :: forall (f :: * -> *). Monad f => Term -> f [[Eqn]]

female :: forall (f :: * -> *). Monad f => Term -> f [[Eqn]]

parent :: forall (f :: * -> *). Monad f => (Term, Term) -> f [[Eqn]]

father :: forall (f :: * -> *). Monad f => (Term, Term) -> f [[Eqn]]

In [22]:
import Control.Monad.Trans.State

newVar :: State Int Term
newVar = do modify (+1)
            i <- get
            return $ V('_':show (i::Int))

In [23]:
grandpa(g,y) = do p <- newVar
                  parent(p,y) .&. father(g,p)

In [24]:
:type grandpa

grandpa :: (Term, Term) -> StateT Int Identity [[Eqn]]

In [25]:
(ess,i) = runState ( father(V"x",bob) ) 0
-- ess
-- mapM_ print ess
i
result = solve ess
mapM_ print [sigma | ([],sigma) <- result]

0

[("x",T "tim" [])]

In [26]:
(ess,i) = runState ( grandpa(V"x",V"y") ) 0
-- ess
-- mapM_ print ess
i
result = solve ess
mapM_ print [sigma | ([],sigma) <- result]

1

[("x",T "john" []),("y",T "bob" []),("_1",T "tim" [])]

In [27]:
ancestor(a,x) = parent(a,x)
                .|.
                do p <- newVar
                   parent(p,x) .&. ancestor(a,p)

In [28]:
(ess,i) = runState ( ancestor(V"x",bob) ) 0
-- ess
-- mapM_ print $ take 20 ess
-- i
result = solve $ take 1000 ess
mapM_ print [sigma | ([],sigma) <- result]

[("x",T "tim" [])]
[("x",T "jane" [])]
[("x",T "john" []),("_1",T "tim" [])]
[("x",T "mary" []),("_1",T "tim" [])]

----
## HW04 (총 12점)

늑대(Wolf), 염소(Goat), 양배추(Cabbage)를 농부가 배를 몰아 강의 동편에서 서편으로 옮기는데
농부는 배에 한번에 하나의 동물 또는 식물만 실어나를 수 있다.
농부의 배가 함께 있는 쪽에서는 어떤 동식물이 강가에 있어도 문제가 없지만
농부가 배를 반대쪽으로 몰아갔을 때는 눈치를 보지 않고 늑대는 염소를 잡아먹고 염소는 양배추를 씹어먹는 위험성이 있다.
이런 위험한 상황이 생기지 않도록 모두가 안전하게 강의 동편에서 시작해 강의 서편으로 이동하려면 어떤 순서로 실어날라야 하는지를 찾는 문제이다.

이 문제를 프롤로그 규칙으로 표현한 프로그램이 우리 과목 저장소 `part2/wgf.pl`에 있다. 이를 하스켈로 옮겨보고 프로그래밍해 보는 것이 이번 과제이다.

위에서 likes나 parent같은 것을 옮길 때와 마찬거지로,
일단 여러 개의 절(clause 즉 "`규칙이름(...) :- 조건.`" 형태)로 이루어진 규칙을 하나의 절로 합친 좀더 원시적인 형태의 프롤로그 정의로 바꿔본 다음에,
그것을 위에서 정의한 `(.=.)`, `(.&.)`, `(.|.)` 연산자 및 새로운 변수가 필요할 경우 `newVar`도 활용해 Haskell로 옮기면 된다.

#### 우선 간단한 `start`, `final`, `opp` 부터 옮겨보고 각각을 `runState`와 `solve`를 이용해 실행하는 예를 하나씩 작성해 보라 (1점 = 0.25 + 0.25 + 0.5)

In [29]:
w = T "w" []
e = T "e" []
s(a,b,c,d) = T "s" [a,b,c,d]

start(x) = undefined

final(x) = undefined

opp(x,y) = undefined

In [30]:
-- 각각의 실행 예를 하나씩 작성하라 (이것까지 해야 각각 0.25점, 0.25점, 0.5점. 정의를 올바르게 옮겼어도 실행 예를 작성하지 않으면 0점임.)


#### `move`와 `safe`를 옮겨보라 (2점)

In [31]:
move(s1,s2) = undefined

safe(s) = undefined

#### 프롤로그에서 다음 과정에 해당하는 실행 예를 각각 작성해 보라 (3점. 각 0.5점)
 * `start(S0), move(S0,S1), safe(S1).`
 * `start(S0), move(S0,S1), safe(S1), move(S1,S2), safe(S2).`
 * `start(S0), move(S0,S1), safe(S1), move(S1,S2), safe(S2), move(S2,S3), safe(S3), move(S3,S4), safe(S4), move(S4,S5), safe(S5), move(S5,S6), safe(S6).`
 * `start(S0), move(S0,S1), safe(S1), move(S1,S2), safe(S2), move(S2,S3), safe(S3), move(S3,S4), safe(S4), move(S4,S5), safe(S5), move(S5,S6), safe(S6), final(S6).`
 * `start(S0), move(S0,S1), safe(S1), move(S1,S2), safe(S2), move(S2,S3), safe(S3), move(S3,S4), safe(S4), move(S4,S5), safe(S5), move(S5,S6), safe(S6), move(S6,S7), safe(S7)`
 * `start(S0), move(S0,S1), safe(S1), move(S1,S2), safe(S2), move(S2,S3), safe(S3), move(S3,S4), safe(S4), move(S4,S5), safe(S5), move(S5,S6), safe(S6), move(S6,S7), safe(S7), final(S7).`

#### 사실 이정도 프로그램은 Haskell 프로그래밍에 어느 정도 익숙하다면 굳이 Prolog로 풀고 나서 옮기지 않아도 Haskell로 직접 풀어볼 수도 있다. 한번 도전해보라. (5점)
  * `saf` 작성 (1점) 위에서 `safe`에 해당
  * `mov` 작성 (2점) 위에서 `move`에 해당
  * `findPath` 작성 (3점). 예를 들어 `findPathsWithLimit 7 (W,W,W,W)`을 입력하면 길이 7이하의 가능한 모든 문제풀이방법을 탐색하도록 작성. (즉 트리에서 리트스들을 뽑아냄)

In [32]:
import Control.Monad

data Pos = W | E deriving (Eq,Ord,Show)
type BWGC = (Pos,Pos,Pos,Pos) -- (B,W,G,C)

data Tree a = Node a [Tree a] deriving (Eq,Ord,Show)

op :: Pos -> Pos
op W = E
op E = W

mov :: BWGC -> [BWGC]
mov s = undefined

saf :: BWGC -> Bool
saf s = undefined

treeFrom :: BWGC -> Tree BWGC
treeFrom s@(E,E,E,E) = Node s []
treeFrom s = Node s [treeFrom s' | s' <- mov s, saf s']

takeTree n (Node s ts)
       | n < 1     = Node s []
       | otherwise = Node s $ map (takeTree (n-1)) ts

-- s로부터 길이 n이하의 문제풀이방법들 탐색. 힌트 끝이 (E,E,E,E) 인 경로들을 모으면 됨
findPathsWithLimit :: Int -> BWGC -> [ [BWGC] ]
findPathsWithLimit n s = findPaths t
  where
  t = takeTree n $ treeFrom s

findPaths :: Tree BWGC -> [ [BWGC] ]
findPaths t = undefined

In [33]:
-- 이렇게 정답 확인 가능
mapM_ print $ findPathsWithLimit 7 (W,W,W,W)

: 